# Homework 2024-11-20


In [1]:
import polars as pl
from scipy import stats as sps


## Datasets


### Student Performance (Math)

- Usage: t-distribution test
- Source: https://archive.ics.uci.edu/dataset/320/student+performance


In [2]:
student_performance_df = pl.read_csv("student-mat.csv", separator=";")

In [3]:
student_performance_df

school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
str,str,i64,str,str,str,i64,i64,str,str,str,str,i64,i64,i64,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""GP""","""F""",18,"""U""","""GT3""","""A""",4,4,"""at_home""","""teacher""","""course""","""mother""",2,2,0,"""yes""","""no""","""no""","""no""","""yes""","""yes""","""no""","""no""",4,3,4,1,1,3,6,5,6,6
"""GP""","""F""",17,"""U""","""GT3""","""T""",1,1,"""at_home""","""other""","""course""","""father""",1,2,0,"""no""","""yes""","""no""","""no""","""no""","""yes""","""yes""","""no""",5,3,3,1,1,3,4,5,5,6
"""GP""","""F""",15,"""U""","""LE3""","""T""",1,1,"""at_home""","""other""","""other""","""mother""",1,2,3,"""yes""","""no""","""yes""","""no""","""yes""","""yes""","""yes""","""no""",4,3,2,2,3,3,10,7,8,10
"""GP""","""F""",15,"""U""","""GT3""","""T""",4,2,"""health""","""services""","""home""","""mother""",1,3,0,"""no""","""yes""","""yes""","""yes""","""yes""","""yes""","""yes""","""yes""",3,2,2,1,1,5,2,15,14,15
"""GP""","""F""",16,"""U""","""GT3""","""T""",3,3,"""other""","""other""","""home""","""father""",1,2,0,"""no""","""yes""","""yes""","""no""","""yes""","""yes""","""no""","""no""",4,3,2,1,2,5,4,6,10,10
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""MS""","""M""",20,"""U""","""LE3""","""A""",2,2,"""services""","""services""","""course""","""other""",1,2,2,"""no""","""yes""","""yes""","""no""","""yes""","""yes""","""no""","""no""",5,5,4,4,5,4,11,9,9,9
"""MS""","""M""",17,"""U""","""LE3""","""T""",3,1,"""services""","""services""","""course""","""mother""",2,1,0,"""no""","""no""","""no""","""no""","""no""","""yes""","""yes""","""no""",2,4,5,3,4,2,3,14,16,16
"""MS""","""M""",21,"""R""","""GT3""","""T""",1,1,"""other""","""other""","""course""","""other""",1,1,3,"""no""","""no""","""no""","""no""","""no""","""yes""","""no""","""no""",5,5,3,3,3,3,3,10,8,7


Reserve only the columns we need for this homework.

> Dataset 只有兩所學校 (「GP」和「MS」）。分數為連續尺度，故適合 t 分配。資料欄位只剩 school、sex（F、M）和分數（G1、G2、G3）。每一列代表一位學生。


In [4]:
student_performance_df = student_performance_df.select("school", "sex", "G1", "G2", "G3")

In [5]:
SexEnum = pl.Enum(["M", "F"])
SchoolEnum = pl.Enum(["GP", "MS"])

student_performance_df = student_performance_df.with_columns(
    pl.col("school").cast(SchoolEnum),
    pl.col("sex").cast(SexEnum),
)

In [6]:
student_performance_df

school,sex,G1,G2,G3
enum,enum,i64,i64,i64
"""GP""","""F""",5,6,6
"""GP""","""F""",5,5,6
"""GP""","""F""",7,8,10
"""GP""","""F""",15,14,15
"""GP""","""F""",6,10,10
…,…,…,…,…
"""MS""","""M""",9,9,9
"""MS""","""M""",14,16,16
"""MS""","""M""",10,8,7


## dyb15-5

- Usage: $\chi^2$ test, independence test


In [7]:
chi_independence_df = pl.read_csv("dyb15-5.csv")

In [8]:
ProgramEnum = pl.Enum(["運動", "電影", "新聞"])
IncomeEnum = pl.Enum(["低", "中", "高"])

chi_independence_df = chi_independence_df.with_columns(
    pl.col("program").replace_strict({1: "運動", 2: "電影", 3: "新聞"}).cast(ProgramEnum),
    pl.col("income").replace_strict({1: "低", 2: "中", 3: "高"}).cast(IncomeEnum),
)

In [9]:
chi_independence_df

income,program,count
enum,enum,i64
"""低""","""運動""",143
"""低""","""電影""",70
"""低""","""新聞""",37
"""中""","""運動""",90
"""中""","""電影""",67
"""中""","""新聞""",43
"""高""","""運動""",17
"""高""","""電影""",13
"""高""","""新聞""",20


## Area, Age, and Gender Statistical table (19CoV)

- Usage: $\chi^2$ test, goodness-of-fit test
- Source: https://data.cdc.gov.tw/zh_TW/dataset/aagstable-19cov


In [10]:
covid19_df = pl.read_excel("Age_County_Gender_19CVS_big5.xlsx")

In [11]:
covid19_df

確定病名,發病年份,發病月份,縣市,鄉鎮,性別,是否為境外移入,年齡層,確定病例數
str,i64,i64,str,str,str,str,str,i64
"""嚴重特殊傳染性肺炎""",2023,1,"""新北市""","""永和區""","""F""","""否""","""70+""",1
"""嚴重特殊傳染性肺炎""",2023,1,"""新北市""","""瑞芳區""","""M""","""否""","""70+""",1
"""嚴重特殊傳染性肺炎""",2023,1,"""桃園市""","""八德區""","""M""","""否""","""70+""",1
"""嚴重特殊傳染性肺炎""",2023,1,"""南投縣""","""埔里鎮""","""F""","""否""","""65-69""",1
"""嚴重特殊傳染性肺炎""",2023,1,"""南投縣""","""埔里鎮""","""F""","""否""","""70+""",1
…,…,…,…,…,…,…,…,…
"""嚴重特殊傳染性肺炎""",2024,8,"""台東縣""","""長濱鄉""","""M""","""否""","""70+""",1
"""嚴重特殊傳染性肺炎""",2024,8,"""台東縣""","""關山鎮""","""F""","""否""","""70+""",1
"""嚴重特殊傳染性肺炎""",2024,8,"""台東縣""","""關山鎮""","""M""","""否""","""70+""",1


考慮到「適合度檢定」只需要「性別」、「確定病例數」這兩個欄位的匯總值，故將資料匯總成「性別」、「這個性別的確定總病例數」兩欄。


In [12]:
covid19_df = covid19_df.group_by("性別").agg(pl.sum("確定病例數"))

In [13]:
covid19_df

性別,確定病例數
str,i64
"""U""",3
"""F""",17370
"""M""",26009


考慮到不需要未知值 (`U`)，故只留 `M` 和 `F` 兩個性別。


In [ ]:
covid19_df = covid19_df.filter(pl.col("性別") != "U")

In [15]:
covid19_df

性別,確定病例數
str,i64
"""F""",17370
"""M""",26009


## $t$-distribution


### 獨立樣本


我想知道 GP 學校中，不同性別間的期初成績 (G1) 是否有顯著差異。

$$
\begin{cases}
H_0 : \mu_{\text{F}} = \mu_{\text{M}} \\
H_1 : \mu_{\text{F}} \neq \mu_{\text{M}}
\end{cases}
$$


In [16]:
male_score = student_performance_df.filter(pl.col("sex") == "M")["G1"]
female_score = student_performance_df.filter(pl.col("sex") == "F")["G1"]

判斷樣本同質性顯著與否，假設如下：

$$
\begin{cases}
H_0 : \sigma_{\text{F}} = \sigma_{\text{M}} \\
H_1 : \sigma_{\text{F}} \neq \sigma_{\text{M}}
\end{cases}
$$


In [17]:
levene = sps.levene(male_score, female_score)

print(levene)
print("樣本同值性顯著嗎？", levene.pvalue <= 0.05)

LeveneResult(statistic=np.float64(0.49025086867044526), pvalue=np.float64(0.4842296949092353))
樣本同值性顯著嗎？ False


考慮到「Levene 檢定」之 $p=0.484>(\alpha=0.05)$，故 not reject $H_0$，變異數相等。


In [18]:
ttest = sps.ttest_ind(male_score, female_score, equal_var=levene.pvalue > 0.05)

print(ttest)
print("reject H0?", ttest.pvalue <= 0.05)  # type: ignore

TtestResult(statistic=np.float64(1.8283673434285317), pvalue=np.float64(0.06825227168840962), df=np.float64(393.0))
reject H0? False


發現 $t=1.83$, $p=0.068>(\alpha=0.05)$，故 not reject $H_0$。


結論：在 95% 的信賴水準下，GP 學校中兩性別間的期初成績無顯著差異。


### 相依樣本


我想知道 GP 學校中，同個人的期初成績 (G1) 和期末成績 (G3) 是否有顯著差異，方便評估學習成效。

$$
\begin{cases}
H_0 : \mu_{\text{G1}} = \mu_{\text{G3}} \\
H_1 : \mu_{\text{G1}} \neq \mu_{\text{G3}}
\end{cases}
$$


In [19]:
g1_score = student_performance_df["G1"]
g3_score = student_performance_df["G3"]

In [20]:
sps.ttest_rel(g1_score, g3_score)

TtestResult(statistic=np.float64(3.5517031247185855), pvalue=np.float64(0.0004290673865804163), df=np.int64(394))

可以發現 $t=3.55$, $p=0.0004<(\alpha=0.05)$，故 reject $H_0$。


結論：在 95% 的信賴水準下，GP 學校中同個人的期初成績和期末成績有顯著差異。


## $\chi^2$-distribution


### 獨立性檢定


In [21]:
chi_independence_df

income,program,count
enum,enum,i64
"""低""","""運動""",143
"""低""","""電影""",70
"""低""","""新聞""",37
"""中""","""運動""",90
"""中""","""電影""",67
"""中""","""新聞""",43
"""高""","""運動""",17
"""高""","""電影""",13
"""高""","""新聞""",20


考慮到 `program` 有兩個以上的類別，且 `count` 為⼀非連續屬性，故適合卡⽅檢定。


假設如下：

- $H_0$：節目型態與收入沒有關係
- $H_1$：節目型態與收入有關係


In [22]:
# 轉換成樞紐表

observed = chi_independence_df.pivot("program", index="income")

In [23]:
observed

income,運動,電影,新聞
enum,i64,i64,i64
"""低""",143,70,37
"""中""",90,67,43
"""高""",17,13,20


In [24]:
sps.chi2_contingency(observed.drop("income"))  # 移除 index

Chi2ContingencyResult(statistic=np.float64(21.173666666666666), pvalue=np.float64(0.0002925239725081441), dof=4, expected_freq=array([[125.,  75.,  50.],
       [100.,  60.,  40.],
       [ 25.,  15.,  10.]]))

發現 $\chi^2=21.1736$, $p=0.0002<(\alpha=0.05)$，故 reject $H_0$。


結論：在 95% 的信賴水準下，節目型態與收入有關係。


### 適合度檢定


In [25]:
covid19_df

性別,確定病例數
str,i64
"""F""",17370
"""M""",26009


因為「病例數」屬於離散資料，故適合卡方適合度檢定。
性別「M」為男性、「F」為女性。


假設如下：

- $H_0$：確定病例數間沒有顯著差異
- $H_1$：確定病例數間有顯著差異


In [26]:
sps.chisquare(covid19_df["確定病例數"])

Power_divergenceResult(statistic=np.float64(1720.4712187925034), pvalue=np.float64(0.0))

發現 $\chi^2=1720.47$, $p=0.0\leq(\alpha=0.05)$，故 reject $H_0$.


結論：在 95% 的信賴水準下，確定病例數間有顯著差異。根據表格，這代表「**男性感染率比女性高**」。
